# Getting started with flex flow


**Learning Objectives** - Upon completing this tutorial, you should be able to:

- Write LLM application using notebook and visualize the trace of your application.
- Convert the application into a flow and batch run against multi lines of data.


## 0. Install dependent packages

In [1]:
%%capture --no-stderr
%pip install -r ./requirements.txt

## 1. Trace your application with promptflow

Assume we already have a python function that calls Unify AI. 

In [2]:
with open("llm.py") as fin:
    print(fin.read())

import os

from dotenv import load_dotenv
from unify import Unify

from promptflow.tracing import trace


@trace
def my_llm_tool(
    prompt: str,
    # for Unify AI, Model and Provider are to be specified by user.
    model_name: str,
    provider_name: str,
    max_tokens: int = 120,
    temperature: float = 1.0,
) -> str:
    if "UNIFY_AI_API_KEY" not in os.environ:
        # load environment variables from .env file
        load_dotenv()

    if "UNIFY_AI_API_KEY" not in os.environ:
        raise Exception("Please specify environment variables: UNIFY_AI_API_KEY")
    messages = [{"content": prompt, "role": "system"}]
    api_key = os.environ.get("UNIFY_AI_API_KEY", None)
    unify_client = Unify(
        api_key=api_key,
        model=model_name,
        provider=provider_name,
    )
    response = unify_client.generate(
        messages=messages,
        max_tokens=int(max_tokens),
        temperature=float(temperature),
    )

    # get first element because prompt is single.
   

Note: before running below cell, please configure required environment variable `UNIFY_AI_API_KEY` by create an `.env` file. Please refer to `./.env.example` as an template.

In [3]:
# Define provider and model from Unify AI. Refer to https://unify.ai/benchmarks
model_name = "llama-3.1-8b-chat"
provider_name = "together-ai"

In [4]:

from llm import my_llm_tool

# pls configure `UNIFY_AI_API_KEY` environment variable
result = my_llm_tool(
    prompt="Write a simple Hello, world! python program that displays the greeting message when executed. Output code only.",
    model_name=model_name,
    provider_name=provider_name,
)
result

'```\nprint("Hello, world!")\n```'

### Visualize trace by using start_trace

Note we add `@trace` in the `my_llm_tool` function, re-run below cell will collect a trace in trace UI.

In [5]:
from promptflow.tracing import start_trace

# start a trace session, and print a url for user to check trace
start_trace()
# rerun the function, which will be recorded in the trace
result = my_llm_tool(
    prompt="Write a simple Hello, world! program that displays the greeting message when executed. Output code only.",
    model_name=model_name,
    provider_name=provider_name,
)
result

Prompt flow service has started...


'```c\n#include <stdio.h>\n\nint main() {\n    printf("Hello, world!\\n");\n    return 0;\n}\n```'

Now, let's add another layer of function call. In `programmer.py` there is a function called `write_simple_program`, which calls a new function called `load_prompt` and previous `my_llm_tool` function.

In [6]:
# show the programmer.py content
with open("programmer.py") as fin:
    print(fin.read())

from pathlib import Path
from typing import TypedDict

from jinja2 import Template
from llm import my_llm_tool

from promptflow.tracing import trace

BASE_DIR = Path(__file__).absolute().parent


class Result(TypedDict):
    output: str


@trace
def load_prompt(jinja2_template: str, text: str) -> str:
    """Load prompt function."""
    with open(BASE_DIR / jinja2_template, "r", encoding="utf-8") as f:
        prompt = Template(
            f.read(), trim_blocks=True, keep_trailing_newline=True
        ).render(text=text)
        return prompt


@trace
def write_simple_program(
    text: str = "Hello World!",
    model_name="llama-3.1-8b-chat",
    provider_name="together-ai",
) -> Result:
    """Ask LLM to write a simple program."""
    prompt = load_prompt("hello.jinja2", text)
    output = my_llm_tool(
        prompt=prompt,
        model_name=model_name,
        provider_name=provider_name,
        max_tokens=120,
    )
    return Result(output=output)


if __name__ == "__main__":


In [7]:
# call the flow entry function
from programmer import write_simple_program

result = write_simple_program("Java Hello, world!")
result

You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=unify-llm-tool&uiTraceId=0x118d5e05554225344dbaed186e3d4d58
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=unify-llm-tool&uiTraceId=0x21c584fafd6393d845c8dc9bad4e5407
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=unify-llm-tool&uiTraceId=0x7f54b359caf9d68702ddfbcd3c88a0b0
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=unify-llm-tool&uiTraceId=0x6dcd0a510e1171662be3e16f98c0b59d


{'output': '```java\npublic class HelloWorld {\n  public static void main(String[] args) {\n    System.out.println("Hello, world!");\n  }\n}\n```'}

### Setup model configuration with environment variables

When used in local, create a model configuration object with environment variables.

Note: before running below cell, please configure required environment variable `UNIFY_AI_API_KEY` and `UNIFY_AI_BASE_URL` by creating a `.env` file. Please refer to `./.env.example` as an template.

Please refer https://unify.ai/docs/concepts/unify_api.html to get Unify AI Api base url.

In [8]:
import os
from dotenv import load_dotenv

from promptflow.core import OpenAIModelConfiguration

# pls configure `UNIFY_AI_API_KEY`, `UNIFY_AI_BASE_URL` environment variables first
if "UNIFY_AI_API_KEY" not in os.environ:
    # load environment variables from .env file
    load_dotenv()

if "UNIFY_AI_API_KEY" not in os.environ:
    raise Exception("Please specify environment variables: UNIFY_AI_API_KEY")
model_config = OpenAIModelConfiguration(
    base_url=os.environ["UNIFY_AI_BASE_URL"],
    api_key=os.environ["UNIFY_AI_API_KEY"],
    model=f"{model_name}@{provider_name}" 
)

### Eval the result 

In [9]:
%load_ext autoreload
%autoreload 2

from code_quality_unify_ai import CodeEvaluator


evaluator = CodeEvaluator(model_config=model_config)
eval_result = evaluator(result)
eval_result


{'correctness': 5,
 'readability': 5,
 'explanation': "The code is correct as it is a valid Java program that prints 'Hello, world!' to the console. The readability is also good as the code is properly formatted and follows standard Java conventions."}

## 2. Batch run the function as flow with multi-line data

Create a [flow.flex.yaml](https://github.com/microsoft/promptflow/blob/main/examples/flex-flows/basic/flow.flex.yaml) file to define a flow which entry pointing to the python function we defined.


In [10]:
# show the flow.flex.yaml content
with open("flow.flex.yaml") as fin:
    print(fin.read())

$schema: https://azuremlschemas.azureedge.net/promptflow/latest/flow.schema.json
entry: programmer:write_simple_program
environment:
  python_requirements_txt: requirements.txt
sample:
  inputs:
    text: Java Hello World!



### Batch run with a data file (with multiple lines of test data)


In [11]:
from promptflow.client import PFClient

pf = PFClient()

In [12]:
data = "./data.jsonl"  # path to the data file
# create run with the flow function and data
base_run = pf.run(
    flow=write_simple_program,
    data=data,
    column_mapping={
        "text": "${data.text}",
    },
    stream=True,
)

[2024-08-04 20:21:05 +0000][promptflow][WARNING] - Found existing /workspaces/promptflow/examples/flex-flows/unify-llm-tool/flow.flex.yaml, will not respect it in runtime.


Prompt flow service has started...
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=unify_llm_tool_20240804_202105_244263


[2024-08-04 20:21:05 +0000][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run unify_llm_tool_20240804_202105_244263, log path: /root/.promptflow/.runs/unify_llm_tool_20240804_202105_244263/logs.txt


You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=unify-llm-tool&uiTraceId=0x5722db5f4be14fd142dd9e29a88ceb10
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=unify-llm-tool&uiTraceId=0x42c6e097bf36c67f3011ac77c96a5ab9
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=unify-llm-tool&uiTraceId=0x079c333a41296e7abe232f38dea00b96
2024-08-04 20:21:09 +0000   42488 execution.bulk     INFO     Process 42539 terminated.
2024-08-04 20:21:09 +0000   42488 execution.bulk     WARNING  Process 42548 had been terminated.
2024-08-04 20:21:09 +0000   42488 execution.bulk     INFO     Process 42533 terminated.
2024-08-04 20:21:05 +0000   42203 execution.bulk     INFO     Current thread is not main thread, skip signal handler registration in BatchEngine.
2024-08-04 20:21:05 +0000   42203 execution.bulk     INFO     Set process count to 3 b

In [13]:
details = pf.get_details(base_run)
details.head(10)

,inputs.text,inputs.line_number,inputs.model_name,inputs.provider_name,outputs.output
0,Python Hello World!,0,llama-3.1-8b-chat,together-ai,"```\n# Print Hello, World! to the console\npri..."
1,C Hello World!,1,llama-3.1-8b-chat,together-ai,```c\n#include <stdio.h>\n\nint main() {\n ...
2,C# Hello World!,2,llama-3.1-8b-chat,together-ai,```csharp\nusing System;\n\nclass HelloWorld \...


## 3. Evaluate your flow
Then you can use an evaluation method to evaluate your flow. The evaluation methods are also flows which usually using LLM assert the produced output matches certain expectation. 

### Run evaluation on the previous batch run
The **base_run** is the batch run we completed in step 2 above, for web-classification flow with "data.jsonl" as input.

In [14]:
# we can also run flow pointing to code evaluator yaml file
eval_flow = "./code-eval-flow.flex.yaml"

eval_run = pf.run(
    flow=eval_flow,
    init={"model_config": model_config},
    data="./data.jsonl",  # path to the data file
    run=base_run,  # specify base_run as the run you want to evaluate
    column_mapping={
        "code": "${run.outputs.output}",
    },
    stream=True,
)

Prompt flow service has started...
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=unify_llm_tool_20240804_202110_069589


[2024-08-04 20:21:10 +0000][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run unify_llm_tool_20240804_202110_069589, log path: /root/.promptflow/.runs/unify_llm_tool_20240804_202110_069589/logs.txt


You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=unify-llm-tool&uiTraceId=0x631051c28f5c5d37c0e5d01fa84c93af
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=unify-llm-tool&uiTraceId=0x5138a969d80113a32a121c0fd1d12980
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=unify-llm-tool&uiTraceId=0x0e136f3212c7562a107e6339be97a5b1
2024-08-04 20:21:15 +0000   42730 execution.bulk     INFO     Process 42770 terminated.
2024-08-04 20:21:15 +0000   42730 execution.bulk     INFO     Process 42786 terminated.
2024-08-04 20:21:15 +0000   42730 execution.bulk     WARNING  Process 42776 had been terminated.
2024-08-04 20:21:10 +0000   42203 execution.bulk     INFO     Current thread is not main thread, skip signal handler registration in BatchEngine.
2024-08-04 20:21:10 +0000   42203 execution.bulk     INFO     Set process count to 3 b

In [15]:
details = pf.get_details(eval_run)
details.head(10)

,inputs.code,inputs.line_number,outputs.correctness,outputs.readability,outputs.explanation
0,"```\n# Print Hello, World! to the console\npri...",0,5,5,The code is correct as it is a simple print st...
1,```c\n#include <stdio.h>\n\nint main() {\n ...,1,5,4,The code is correct as it is a simple C progra...
2,```csharp\nusing System;\n\nclass HelloWorld \...,2,5,4,"The code is correct as it is a simple 'Hello, ..."


In [16]:
import json

metrics = pf.get_metrics(eval_run)
print(json.dumps(metrics, indent=4))

{
    "average_correctness": 5.0,
    "average_readability": 4.333333333333333,
    "total": 3
}


In [17]:
pf.visualize([base_run, eval_run])

Prompt flow service has started...
The HTML file is generated at '/tmp/pf-visualize-detail-azfuci3g.html'.
Trying to view the result in a web browser...
Successfully visualized from the web browser.


Loading "original-fs" failed
Error: Cannot find module 'original-fs'
Require stack:
- /vscode/vscode-server/bin/linux-x64/b1c0a14de1414fcdaa400695b4db1c0799bc3124/out/server-cli.js
    at Module._resolveFilename (node:internal/modules/cjs/loader:1145:15)
    at Module._load (node:internal/modules/cjs/loader:986:27)
    at Module.require (node:internal/modules/cjs/loader:1233:19)
    at require (node:internal/modules/helpers:179:18)
    at i (/vscode/vscode-server/bin/linux-x64/b1c0a14de1414fcdaa400695b4db1c0799bc3124/out/server-cli.js:3:98)
    at r.load (/vscode/vscode-server/bin/linux-x64/b1c0a14de1414fcdaa400695b4db1c0799bc3124/out/server-cli.js:2:1637)
    at h.load (/vscode/vscode-server/bin/linux-x64/b1c0a14de1414fcdaa400695b4db1c0799bc3124/out/server-cli.js:1:13958)
    at u (/vscode/vscode-server/bin/linux-x64/b1c0a14de1414fcdaa400695b4db1c0799bc3124/out/server-cli.js:3:9338)
    at Object.errorback (/vscode/vscode-server/bin/linux-x64/b1c0a14de1414fcdaa400695b4db1c0799bc3124/o

## End Note

By now you've successfully run your simple code generation and evaluation using Unify AI.